Business Understanding: **Deseja-se otimizar apostas esportivas em futebol através de um algoritmo que classifica se uma aposta é vencedora ou perdedora.**

Dados a serem coletados: 
* Lista de jogadores e tecnicos que atuam nas séries A, B, C e Feminino no Brasil e suas posições.
* Resultados de todos os jogos
* Notas de jogadores
* Estatísticas de jogos (Posse de bola, chutes a gol, cartões, etc)
* Percepção da torcida e noticiários.
* Odds
* Posição e informações do campeonato
* Estreia de técnico ou de jogador
* Clima e horário das partidas
* Jogos em casa e fora de casa


Como o modelo será construído, inicialmente, como um classificador que irá indicar as seguintes categorias: Time A vence, Time A perde ou Time A empata.

Os dados sobre os jogadores serão alimentados da seguinte forma:
* Para cada posição, haverá 4 dados numéricos sobre notas: Nota no último jogo, média móvel dos últimos 5 jogos e média móvel dos últimos 10 jogos e alfa para a regressão sobre as notas.
* Para cada posição haverá 4 dados numéricos sobre gols: Número de gols no campeonato/jogos, número de gols no ano/jogos, número de gols nos últimos 5 jogos/jogos, número de gols nos últimos 10 jogos/jogos.
* Para cada posição haverá 2 dados relacionados à cartões: Número de cartões vermelhos no ano/jogos, número de cartões amarelos/jogos, pendurado ou não.


Os dados para o time serão alimentados da seguinte forma:
* Para cada time: Resultado do último jogo, número de vitórias nos últimos 5 jogos, número de vitórias nos últimos 10 jogos, alfa da regressão de vitórias em 5 e 10 jogos, número de gols por jogo no último jogo, nos últimos 5 jogos/jogos e nos últimos 10 jogos/jogos, alfa da regressão para o número de gols.
* Para cada time: Estreia de novo jogador e de novo técnico.
* Para cada time: Posse de bola média nos últimos 5 jogos e 10 jogos, chutes a gol médio no último jogo e nos últimos 5 jogos.
* Para cada time: Número de jogadores expulsos nos últimos 5, 10 e 20 jogos.
* Para cada time: Número de jogadores convocados para seleções.
* Local do jogo: Dentro ou fora de casa, estádio, clima e horário.
* Para cada time: Posição no campeonato.
* Para cada time: Percepção da torcida sobre o time (Positiva ou negativa), percepção da imprensa (Positiva ou negativa), odds média entre todas as casas (OdsPedia), odds máxima de vitória, odds mínima de derrota, odds máxima de vitória do time B e odds mínima de vitória do time B.
* Para cada time: Gols contra campeonato/ gols a favor.

<h1> Coletando a lista de jogadores para todos os times da Série A </h1>

In [1]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import selenium
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium.webdriver.support.ui import Select


# url='https://oddspedia.com/br/futebol/brasil/brasileirao-serie-a'
# response = requests.get(url)
# soup = BeautifulSoup(response.content, "html.parser")

Scrapping da página Soccerstats

In [2]:
driver = webdriver.Chrome()
driver.get("https://www.soccerstats.com/latest.asp?league=brazil")

Coletando tabela do campeonato



In [3]:
agree_cookies=driver.find_elements_by_class_name("qc-cmp2-summary-buttons")
try:
    agree_cookies=agree_cookies[0].find_elements_by_css_selector('button')
    for button in agree_cookies:
        if(button.text=='AGREE'):
            button.click()
            break
except:
    next
table_list=driver.find_elements_by_id('btable')
main_table=pd.DataFrame(columns=["Position", "Team", "GP", "W", "D", "L","GF","GA","GD","Pts","G-1","G-2","G-3","G-4","G-5","G-6","PPG","PPG8","CS","FTS"])
for table in table_list:
    if(table.text.find('GP W D L GF GA GD Pts')!=-1):        
        for row in table.find_elements_by_class_name('odd'):
            pos=row.find_elements_by_css_selector('td')[0].text
            team=row.find_elements_by_css_selector('td')[1].text
            gp=row.find_elements_by_css_selector('td')[2].text
            w=row.find_elements_by_css_selector('td')[3].text
            d=row.find_elements_by_css_selector('td')[4].text
            l=row.find_elements_by_css_selector('td')[5].text
            gf=row.find_elements_by_css_selector('td')[6].text
            ga=row.find_elements_by_css_selector('td')[7].text
            gd=row.find_elements_by_css_selector('td')[8].text
            pts=row.find_elements_by_css_selector('td')[9].text
            past_res_obj=row.find_elements_by_css_selector('td')[10]
            g=[]
            for index,item in enumerate(past_res_obj.find_elements_by_css_selector('div')):
                if(index!=0 and index!=7):
                    g.append(item.get_attribute('class'))
            ppg=row.find_elements_by_css_selector('td')[32].text
            ppg8=row.find_elements_by_css_selector('td')[33].text
            cs=row.find_elements_by_css_selector('td')[34].text
            fts=row.find_elements_by_css_selector('td')[35].text
            main_table=main_table.append({"Position":pos, "Team":team, "GP":gp, "W":w, "D":d, "L":l,
                               "GF":gf,"GA":ga,"GD":gd,"Pts":pts,"G-1":g[0],"G-2":g[1],"G-3":g[2],
                               "G-4":g[3],"G-5":g[4],"G-6":g[5],"PPG":ppg,"PPG8":ppg8,"CS":cs,"FTS":fts},ignore_index=True)    
            
            

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:1: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  agree_cookies=driver.find_elements_by_class_name("qc-cmp2-summary-buttons")
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:10: DeprecationWarning: find_elements_by_id is deprecated. Please use find_elements(by=By.ID, value=id_) instead
  table_list=driver.find_elements_by_id('btable')
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:15: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  pos=row.find_elements_by_css_selector('td')[0].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:16: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  team=row.find_elements_by_css_selector

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:17: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  gp=row.find_elements_by_css_selector('td')[2].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:18: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  w=row.find_elements_by_css_selector('td')[3].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:19: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  d=row.find_elements_by_css_selector('td')[4].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:20: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  l=row.find_elements_b

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:24: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  pts=row.find_elements_by_css_selector('td')[9].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:25: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  past_res_obj=row.find_elements_by_css_selector('td')[10]
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:27: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  for index,item in enumerate(past_res_obj.find_elements_by_css_selector('div')):
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_sel

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:30: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg=row.find_elements_by_css_selector('td')[32].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:31: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ppg8=row.find_elements_by_css_selector('td')[33].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:32: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  cs=row.find_elements_by_css_selector('td')[34].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\249910690.py:33: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  fts=row.find_

In [4]:
years=[2022,2021,2020,2019,2018,2017]
matches_table=pd.DataFrame(columns=['Year','Date','Local','Visitor','HG','AG','htHG','htAG'])
for year in years:
    year_selector=driver.find_elements_by_class_name('dropdown-content')
    year_selector=year_selector[0].find_elements_by_css_selector('a')
    for row in year_selector:
        if(row.get_property('text')==str(year)):
            print(row.get_property('text'))
            driver.get(row.get_property('href'))
            break

    buttons_index=driver.find_elements_by_css_selector("td")
    buttons=[]
    for item in buttons_index:
        try:
            item.get_attribute('valign')
            if(item.text.find('MATCHES FORM')!=-1):
                buttons.append(item)
                if(len(buttons)==2):
                    break
        except:
            next

    for button in buttons[1].find_elements_by_css_selector('a'):
        if(button.text=='MATCHES'):
            button.click()
            break
    buttons_index=driver.find_elements_by_css_selector("td")
    buttons=[]
    for item in buttons_index:
        try:
            if(item.text.find('Filter')!=-1):
                buttons.append(item)
                if(len(buttons)==2):
                    break
        except:
            next
    buttons[1].click()
    selection_month=driver.find_elements_by_name('themonth')
    select=Select(selection_month[0])
    select.select_by_visible_text('- any month -')
    search_button=driver.find_elements_by_css_selector('input')
    search_button[2].click()

    game_tables=driver.find_elements_by_class_name('sortable')
    for row in game_tables[0].find_elements_by_css_selector('tr'):
        try:
            date=row.find_elements_by_css_selector('td')[0].text
            local=row.find_elements_by_css_selector('td')[1].text
            visitor=row.find_elements_by_css_selector('td')[2].text
            hg=row.find_elements_by_css_selector('td')[3].text
            ag=row.find_elements_by_css_selector('td')[4].text
            hthg=row.find_elements_by_css_selector('td')[13].text
            htag=row.find_elements_by_css_selector('td')[14].text
            matches_table=matches_table.append({'Year':year,'Date':date,'Local':local,'Visitor':visitor,'HG':hg,'AG':ag,'htHG':hthg,'htAG':htag},ignore_index=True)
        except:
            next
    matches_table=matches_table[:len(matches_table)-2]


C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  year_selector=driver.find_elements_by_class_name('dropdown-content')
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  year_selector=year_selector[0].find_elements_by_css_selector('a')


2022


C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  buttons_index=driver.find_elements_by_css_selector("td")
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:24: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  for button in buttons[1].find_elements_by_css_selector('a'):
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:28: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  buttons_index=driver.find_elements_by_css_selector("td")
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:39: DeprecationWarning: find_elements_by_name is deprecated. Please use find_elements(by=By.NAME, value=name)=By.NAME, value=name) instead
  

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:54: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  htag=row.find_elements_by_css_selector('td')[14].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches_table=matches_table.append({'Year':year,'Date':date,'Local':local,'Visitor':visitor,'HG':hg,'AG':ag,'htHG':hthg,'htAG':htag},ignore_index=True)
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:48: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  date=row.find_elements_by_css_selector('td')[0].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:49: DeprecationWarning: find_elements_by_css_selector is depre

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:49: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  local=row.find_elements_by_css_selector('td')[1].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:50: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  visitor=row.find_elements_by_css_selector('td')[2].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:51: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  hg=row.find_elements_by_css_selector('td')[3].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:52: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ag=row.find

...

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:52: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  ag=row.find_elements_by_css_selector('td')[4].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:53: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  hthg=row.find_elements_by_css_selector('td')[13].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:54: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  htag=row.find_elements_by_css_selector('td')[14].text
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches_table=matc

C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:4: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  year_selector=driver.find_elements_by_class_name('dropdown-content')
C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:5: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  year_selector=year_selector[0].find_elements_by_css_selector('a')


2021


C:\Users\luisf\AppData\Local\Temp\ipykernel_10312\374999775.py:12: DeprecationWarning: find_elements_by_css_selector is deprecated. Please use find_elements(by=By.CSS_SELECTOR, value=css_selector) instead
  buttons_index=driver.find_elements_by_css_selector("td")


IndexError: list index out of range

In [ ]:
matches_table

In [ ]:
year=2022

In [ ]:
main_table

In [128]:
driver.close()

In [ ]:
driver.get("'https://oddspedia.com/br/futebol/brasil/brasileirao-serie-a'")
driver.find_elements(By.'league-aside')